In [1]:
import cv2
import depthai as dai
import numpy as np
import time

In [2]:
# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disparity = False
# Better accuracy for longer distance, fractional disparity 32-levels:
subpixel = False
# Better handling for occlusions:
lr_check = True

# Create pipeline
pipeline = dai.Pipeline()

# camera configurations
camRgb = pipeline.create(dai.node.ColorCamera)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
xoutRgb = pipeline.createXLinkOut()
xoutRgb.setStreamName("rgb")
camRgb.video.link(xoutRgb.input)
left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
left.setBoardSocket(dai.CameraBoardSocket.LEFT)
right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
depth = pipeline.createStereoDepth()
left.out.link(depth.left)
right.out.link(depth.right)
xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
depth.disparity.link(xout.input)

prev_frame_time = 0
new_frame_time = 0
DIM = (720, 480)

In [3]:
with dai.Device(pipeline) as device:

    # Output queue will be used to get the disparity frames from the outputs defined above
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)
    qRgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    while True:

        new_frame_time = time.time()

        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)

        inRgb = qRgb.get()

        inDisparity = q.get()  # blocking call, will wait until a new data has arrived
        frame = inDisparity.getFrame()

        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)

        frame_rgb = cv2.resize(inRgb.getCvFrame(), DIM, interpolation=cv2.INTER_AREA)

        cv2.imshow("rgb", frame_rgb)
        cv2.imshow("disparity", frame)

        print("FPS: ", fps)
        if cv2.waitKey(1) == ord('q'):
            break

FPS:  0
FPS:  2
FPS:  8
FPS:  9
FPS:  10
FPS:  11
FPS:  9
FPS:  11
FPS:  9
FPS:  10
FPS:  11
FPS:  10
FPS:  12
FPS:  54
FPS:  5
FPS:  12
FPS:  12
FPS:  10
FPS:  12
FPS:  2
FPS:  58
FPS:  87
FPS:  101
FPS:  89
FPS:  16
FPS:  10
FPS:  11
FPS:  11
FPS:  10
FPS:  13
FPS:  1
FPS:  76
FPS:  92
FPS:  79
FPS:  73
FPS:  26
FPS:  6
FPS:  11
FPS:  11
FPS:  11
FPS:  9
FPS:  1
FPS:  52
FPS:  92
FPS:  97
FPS:  99
FPS:  96
FPS:  15
FPS:  8
FPS:  9
FPS:  10
FPS:  12
FPS:  11
FPS:  10
FPS:  10
FPS:  8
FPS:  12
FPS:  10
FPS:  9
FPS:  15
FPS:  10
FPS:  9
FPS:  9
FPS:  13
FPS:  9
FPS:  12
FPS:  10
FPS:  9
FPS:  13
FPS:  8
FPS:  12
FPS:  12
FPS:  9
FPS:  13
FPS:  8
FPS:  11
FPS:  14
FPS:  8
FPS:  13
FPS:  9
FPS:  12
FPS:  9
FPS:  17
FPS:  8
FPS:  12
FPS:  10
FPS:  11
FPS:  9
FPS:  11
FPS:  7
FPS:  14
FPS:  9
FPS:  10
FPS:  11
FPS:  10
FPS:  11
FPS:  11
FPS:  11
FPS:  10
FPS:  7
FPS:  11
FPS:  15
FPS:  11
FPS:  10
FPS:  8
FPS:  13
FPS:  9
FPS:  10
FPS:  11
FPS:  16
FPS:  11
FPS:  8
FPS:  10
FPS:  14
FPS:  9